The notebook is simple pytorch efficientnet baseline for six channels input data.I fork @SDSTony notebook,replace resnet18 with efficientnet,and change the first conv layer with pretrained weights,in order to process six channels data.I also delete some useless code compared with original notebook.If the notebook helps you,please upvote this notebook and original notebook,thanks.

# import

In [ ]:
%matplotlib inline
import os,cv2,random,glob
from torch.utils.data.sampler import SubsetRandomSampler
import torch, os
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import GroupKFold, StratifiedKFold
import numpy as np
from tqdm import tqdm
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm
from timm.models.layers.conv2d_same import Conv2dSame
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

# config

In [ ]:
device = "cuda"
epochs = 10

# Dataset

In [ ]:
class ClassificationDataset:
    
    def __init__(self, image_paths, targets, resize=None, augmentations=None): 
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):      
        image = np.load(self.image_paths[item]).astype(float)

        targets = self.targets[item]
        
        if self.resize is not None:
            image = np.transpose(image, (1,2,0))
            image = cv2.resize(image, dsize=self.resize, interpolation=cv2.INTER_CUBIC)        
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.long),
        }

In [ ]:
def train(data_loader, model, optimizer, device):
    
    model.train()
    
    for i,data in enumerate(tqdm(data_loader, position=0, leave=True, desc='Training')):
        inputs = data["image"]
        targets = data['targets']
        
        inputs = inputs.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))
        loss.backward()
        optimizer.step()
        
def evaluate(data_loader, model, device):
    model.eval()
    
    final_targets = []
    final_outputs = []
    
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            inputs = data["image"]
            targets = data["targets"]
            inputs = inputs.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            
            output = model(inputs)
            
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(output)
            
    return final_outputs, final_targets

## Train

In [ ]:
train_labels = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
train_labels['img_path'] = train_labels['id'].apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')

images = train_labels.img_path.values
targets = train_labels.target.values
model=timm.create_model('tf_efficientnet_b0_ns',pretrained=False,num_classes=1000)
model.load_state_dict(torch.load('../input/timm-pretrained-efficientnet/efficientnet/efficientnet_b0_ra-3dd342df.pth'))
model.classifier=nn.Linear(1280,1)
conv_stem_wight=model.conv_stem.weight
conv_stem_bias=model.conv_stem.bias
model.conv_stem = Conv2dSame(6, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
model.conv_stem.weight=torch.nn.Parameter(torch.cat((conv_stem_wight, conv_stem_wight), dim=1))
model.to(device)

train_images, valid_images, train_targets, valid_targets = train_test_split(images, targets, stratify=targets, random_state=42)
train_dataset = ClassificationDataset(image_paths=train_images,
                                     targets=train_targets,
                                     augmentations=None)

train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=16,
                                          shuffle=True,
                                          num_workers=4)

valid_dataset = ClassificationDataset(image_paths=valid_images,
                                     targets=valid_targets,
                                     augmentations=None)

valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=4)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
for epoch in range(epochs):
    train(train_loader, model, optimizer, device=device)
    predictions, valid_targets = evaluate(valid_loader, model, device=device)
    roc_auc = metrics.roc_auc_score(valid_targets, predictions)
    print(f"Epoch={epoch}, Valid ROC AUC={roc_auc}")
    torch.save(model.state_dict(),f'epochs:{epochs},roc_auc{roc_auc}_.pt')

## Inference

In [ ]:
submission = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
submission['img_path'] = submission['id'].apply(lambda x: f'../input/seti-breakthrough-listen/test/{x[0]}/{x}.npy')
test_images = submission.img_path.values
dummy_targets = submission.target.values
test_dataset = ClassificationDataset(image_paths=test_images,
                                     targets=dummy_targets,
                                     augmentations=None)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=4)

predictions, valid_targets = evaluate(test_loader, model, device=device)
# normalize
predictions = np.array(predictions)
predictions = (predictions - predictions.min()) / (predictions.max() - predictions.min())
submission.target = predictions
submission.drop(['img_path'], axis=1, inplace=True)
submission.to_csv('submission.csv', index=False)